In [10]:
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf


In [11]:
recipes = pd.read_csv("../mock_data/mock-data_recipe_10.csv")

In [12]:
recipes.columns

Index(['Unnamed: 0', 'recipe_id', 'recipe_name', 'aver_rate', 'image_url',
       'review_nums', 'ingredients', 'cooking_directions', 'nutritions',
       'reviews'],
      dtype='object')

from https://www.tensorflow.org/tutorials/load_data/tfrecord

In [13]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [14]:
tf.io.read_file("../data/foodRecSys-V1_kaggle/raw-data-images/raw-data-images/222388.jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xdb\x00C\x00\x04\x02\x03\x03\x03\x02\x04\x03\x03\x03\x04\x04\x04\x04\x05\t\x06\x05\x05\x05\x05\x0b\x08\x08\x06\t\r\x0b\r\r\r\x0b\x0c\x0c\x0e\x10\x14\x11\x0e\x0f\x13\x0f\x0c\x0c\x12\x18\x12\x13\x15\x16\x17\x17\x17\x0e\x11\x19\x1b\x19\x16\x1a\x14\x16\x17\x16\xff\xdb\x00C\x01\x04\x04\x04\x05\x05\x05\n\x06\x06\n\x16\x0f\x0c\x0f\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\x16\xff\xc0\x00\x11\x08\x01\x95\x02\xd0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1

In [15]:
def serialize_example(recipe_info, image_dir_path = "../data/foodRecSys-V1_kaggle/raw-data-images/raw-data-images/"):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """

  recipe_id = recipe_info.recipe_id

  # Create a dictionary mapping the feature name to the tf.train.Example-compatible
  # data type.
  feature = {
      "recipe_id": _int64_feature(recipe_id),
      "recipe_name": _bytes_feature(bytes(recipe_info.recipe_name, "utf-8")),
      "recipe_image": _bytes_feature(tf.io.read_file(f"{image_dir_path}{recipe_id}.jpg")),
      "aver_rate": _float_feature(recipe_info.aver_rate),
      "review_nums": _int64_feature(recipe_info.review_nums),
      "ingredients": _bytes_feature(bytes(recipe_info.ingredients, "utf-8")), # replace ^ with maybe
      "cooking_directions": _bytes_feature(bytes(recipe_info.ingredients, "utf-8")),
      "reviews": _bytes_feature(bytes(recipe_info.ingredients, "utf-8")),
      
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  
  return recipe_id, example_proto.SerializeToString()

In [20]:
def create_proto_file(pandas_row, dir_path = "../mock_data/output_proto_files/"):
    
    recipe_id, serialized_example = serialize_example(pandas_row)

    proto_filename = f"{dir_path}{recipe_id}.proto"
    # with tf.io.TFRecordWriter(proto_filename) as writer:
    tf.io.write_file(proto_filename, serialized_example)

    return f"Proto file '{proto_filename}' created."

In [21]:
recipes.apply(create_proto_file, axis = 1)

0    Proto file '../mock_data/output_proto_files/22...
1    Proto file '../mock_data/output_proto_files/24...
2    Proto file '../mock_data/output_proto_files/21...
3    Proto file '../mock_data/output_proto_files/87...
4    Proto file '../mock_data/output_proto_files/24...
5    Proto file '../mock_data/output_proto_files/21...
6    Proto file '../mock_data/output_proto_files/20...
7    Proto file '../mock_data/output_proto_files/24...
8    Proto file '../mock_data/output_proto_files/22...
9    Proto file '../mock_data/output_proto_files/25...
dtype: object

In [22]:
proto_filename = "../mock_data/output_proto_files/87211.proto"

In [26]:
# Read and decode the proto file
raw_dataset = tf.data.TFRecordDataset([proto_filename])

feature_description = {
    "recipe_id": tf.io.FixedLenFeature([], tf.int64),
    "recipe_name": tf.io.FixedLenFeature([], tf.string),
    "recipe_image": tf.io.FixedLenFeature([], tf.string),
    "aver_rate": tf.io.FixedLenFeature([], tf.float32),
    "review_nums": tf.io.FixedLenFeature([], tf.int64),
    "ingredients": tf.io.FixedLenFeature([], tf.string),
    "cooking_directions": tf.io.FixedLenFeature([], tf.string),
    "reviews": tf.io.FixedLenFeature([], tf.string),
}



In [25]:
tf.io.parse_single_example(tf.io.read_file(proto_filename), feature_description)

{'aver_rate': <tf.Tensor: shape=(), dtype=float32, numpy=4.625>,
 'cooking_directions': <tf.Tensor: shape=(), dtype=string, numpy=b'focaccia bread quartered^prepared basil pesto^diced cooked chicken^diced green bell pepper^diced red onion^shredded Monterey Jack cheese'>,
 'ingredients': <tf.Tensor: shape=(), dtype=string, numpy=b'focaccia bread quartered^prepared basil pesto^diced cooked chicken^diced green bell pepper^diced red onion^shredded Monterey Jack cheese'>,
 'recipe_id': <tf.Tensor: shape=(), dtype=int64, numpy=87211>,
 'recipe_image': <tf.Tensor: shape=(), dtype=string, numpy=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xdb\x00C\x00\x04\x02\x03\x03\x03\x02\x04\x03\x03\x03\x04\x04\x04\x04\x05\t\x06\x05\x05\x05\x05\x0b\x08\x08\x06\t\r\x0b\r\r\r\x0b\x0c\x0c\x0e\x10\x14\x11\x0e\x0f\x13\x0f\x0c\x0c\x12\x18\x12\x13\x15\x16\x17\x17\x17\x0e\x11\x19\x1b\x19\x16\x1a\x14\x16\x17\x16\xff\xdb\x00C\x01\x04\x04\x04\x05\x05\x05\n\x06\x06\n\x16\x0f\x0c\x0f\x16\x16\x16